# MNIST CNN klasifikacija

U ovoj svesci bavićemo se, takođe, zadatkom klasifikacije MNIST slika. U pristupu ćemo koristiti konvolutivne neuronske mreže.

Prvo ćemo učitati sve neophodne biblioteke.

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import losses, optimizers

In [5]:
from tensorflow.keras import utils
from tensorflow.keras import backend as K

In [6]:
np.random.seed(7)

## Korak 1: Učitavanje skupa podataka

Da bismo učitali podatke, iskoristićemo podršku Keras biblioteke.

Prilikom obrade slika ulazi u mrežu treba da budu 4D tenzori. Uzimaju se u obzir širina i visina slike, broj kanala (jedan za monohromatske ili tri za RGB slike) i broj instacni. Stoga je učitane podatke potrebno transformisati na ovaj oblik. 

<img src='assets/4D_tensor.png'>

Postoje dve konvencije zapisa slika. `Channels last` zapis je oblika `(samples, height, width, color_depth)`, a `chanell first` oblika `(samples, color_depth, height, width)`. Keras biblioteka (i TensorFlow) podrazumevano koriste `Channel last` zapis. Tako se, na primer, paketić veličine 128 koji sadrži crno-bele slike dimenzija 256x256 opisuje tenzorom dimenzija (128, 256, 256, 1), a recimo, paketić veličine 128 slika koje su u RGB modu tenzorom dimenzija (128, 256, 256, 3).

In [7]:
K.image_data_format()

'channels_last'

Sledeći kod proverava koje od ovih podešavanja važi. U daljem radu se svakako možemo osloniti na podrazumevana podešavanja.

## Korak 2: Priprema podataka

Slike ćemo pripremiti tako što ćemo ih normalizovati, tj. svesti vrednosti pojedinačnih piksela na vrednost iz intervala iz [0, 1]. Prethodno ćemo promeniti tip elemenata matrice sa uint8 na float32 kako bi mogli da sačuvamo realne vrednosti.

Vrednosti ciljne promenljive ćemo pripremiti tako što ćemo ih transformisati u vektore dužine 10 sa jedinicom na poziciji koja odgovara vrednosti cifre (takozvano *one-hot* kodiranje). Na primer, broj 3 će biti transformisan u vektor \[0, 0, 0, 1, 0, 0, 0, 0, 0, 0\] sa jedinicom na poziciji tri. 

## Korak 3: Pravljenje modela

Naš model će se sastojati iz više konvolutivnih i agregacionih slojeva. 

Konvolutivni slojevi `Conv2D` se konfigurišu tako što se prvo navede broj filtera (parametar `filters`, obično neimenovan), zatim veličine kernela (`kernel_size` parametar), veličina pomeraja (parametar `strides`) i uokvirenje (`padding` parametar). <img src='assets/convolution_operation.gif' style='height: 300px'>

Na primer, na slici je prikazan kernel veličine 3x3 koji prolazi kroz ulaz sa horizontalnim i vertikalnim pomerajem veličine 2. Dodati beli skup kvadrata predstavlja uokvirenje i u zavisnosti od njegovog prisustva veličina izlazne slike može biti istih dimenzija (u Keras biblioteci se ovo naglašava vrednošću same) ili nešto manja (u Keras biblioteci se ovo naglašava vrednošću valid). 

Agregacioni slojevi (`MaxPooling2D` i `AvgPooling2D`) vrše redukciju slojeva svođenjem blokova zadatih večina na njihove maksimalne ili prosečne vrednosti. Veličina bloka zadaje se parametrom `pool_size`. <img src='assets/pooling.png' style='width: 300px'>

Prilikom treniranja koristićemo i `Dropout` tehniku regularizacije. Podsetimo se da na ovaj način isključujemo nasumično odabrane neurone i omogućavamo drugačiji protok podataka kroz mrežu. <img src='assets/dropout.png'>

Funkcija `Flatten` se koristi za transformisanje matrica u vektore i obično se koristi kao priprema za dalji gusti deo mreže. 

## Korak 4: Analiza i evaluacija modela

Mrežu ćemo evaluirati na test skupu. 